### Notebook which does some data spotchecking to ensure the data processing didn't introduce any errors

In [1]:
import xarray as xr
from joblib import Parallel, delayed
import pickle
import pandas as pd
from openavalancheproject.prep_ml import PrepML
import numpy as np

In [2]:
interpolate = 1 
data_root = '/media/scottcha/E1/Data/OAPMLData/'

In [3]:
pml = PrepML(data_root, interpolate, date_start='2018-11-01', date_end='2019-04-30', date_train_test_cutoff='2019-04-01')   

In [4]:
#get produced dataset
ml_path = data_root + '/5.MLData/'
num_features = 978
interpolation = 1
label = 'Day1DangerAboveTreeline'
file_label = 'co_Day1DangerAboveTreeline_small'
X = np.load(ml_path + '/X_all_' + file_label + '.npy', mmap_mode='r')



In [5]:
i=0


y_df = None

df = pd.read_parquet(ml_path + '/y_train_batch_' + str(i) + '_' + file_label + '.parquet')  
y_df = pd.concat([y_df, df])

df = pd.read_parquet(ml_path + '/y_test_batch_' + str(i) + '_' + file_label + '.parquet')  
y_df = pd.concat([y_df, df])

y_df = y_df.reset_index(drop=True)

In [6]:
num_files_to_validate = 10
y_to_validate = y_df.sample(n=num_files_to_validate)

In [7]:
def is_equal(a, b):
    return np.array_equiv(np.nan_to_num(a),np.nan_to_num(b))
    

In [9]:
features = pd.read_csv(ml_path + '/FeatureLabels_' + file_label + '.csv')['0']

In [10]:
features

0              UGRD_planetaryboundarylayer_min
1              VGRD_planetaryboundarylayer_min
2             VRATE_planetaryboundarylayer_min
3                             GUST_surface_min
4                                 HGT_10mb_min
                        ...                   
973     HGT_PV_EQ_M2eM06_Km_2_kg_s_surface_avg
974    PRES_PV_EQ_M2eM06_Km_2_kg_s_surface_avg
975    VWSH_PV_EQ_M2eM06_Km_2_kg_s_surface_avg
976                     PRMSL_meansealevel_avg
977                           N5WAVH_500mb_avg
Name: 0, Length: 978, dtype: object

In [11]:
for r in y_to_validate.iterrows():
    i = r[0]
    r = r[1]
    
    test_region = r['UnifiedRegion']
    test_lat = r['latitude']
    test_lon = r['longitude']
    test_lookback = 180
    test_date = r['parsed_date']
    print('Sample: ' + r['sample'])
    data = pml.get_data_zarr(test_region, lat=test_lat, lon=test_lon, lookback_days=test_lookback, date=test_date, variables=features.values)
    data = data.sortby('variable')
    data2 = X[i]
    assert is_equal(data.vars.values, data2)

Sample: 2017-12-29 00:00:00: Aspen Zone
Sample: 2019-01-16 00:00:00: North San Juan Mountains
Sample: 2017-04-10 00:00:00: Sangre de Cristo Range
Sample: 2019-04-11 00:00:00: North San Juan Mountains
Sample: 2017-01-25 00:00:00: Sangre de Cristo Range
Sample: 2018-01-27 00:00:00: North San Juan Mountains
Sample: 2016-02-01 00:00:00: North San Juan Mountains
Sample: 2016-01-21 00:00:00: Vail Summit Zone
Sample: 2015-11-18 00:00:00: South San Juan Mountains
Sample: 2015-11-27 00:00:00: Sangre de Cristo Range
